# TP1: Algoritmos de búsqueda en Torre de Hanoi

## Autores:
- Marcos Dominguez
- Lucas Monzón
- Felipe Sarche


In [ ]:
import sys
import os

# Añadir la ruta a sys.path
sys.path.append(os.path.abspath('../lib/intro_ia/clase2/hanoi_tower'))

# import aima
# import hanoi_states
# import search


1. PEAS del problema

| Agente | Performance | Environment | Actuators | Sensors |
|--------|-------------|-------------|-----------|---------|
|Torre de Hanoi | Utilizar el menor numero de movimientos posibles| Discos y torres  | Mover discos respetando las reglas del juego| Posición de los discos y estado de las torres|

2. Propiedades del Entorno:

a. Totalmente observable: Sabemos exactamente todas las posiciones de los discos y el estado de las torres.
b. Determinístico: Las acciones tienen resultados predecibles.
c. Secuencial: Cada movimiento de un disco afecta el estado del entorno y las opciones disponibles para los movimientos futuros.
d. Estático: No hay cambios de estados mientras que el agente no intervenga.
e. Discreto: Los estados y acciones son finitos y contables.
f. Agente individual: Solo existe un agente realizando acciones en el entorno.

3.
- Estado: Ubicación de los discos en las torres en un momento específico.
- Espacio de estados: Cantidad de posiciones posibles para los discos en las torres.
- Árbol de búsqueda:
- Nodo de búsqueda:
- Objetivo : Configuración en la que todo los discos han sido movidos de la torre inicial a la torre objetivo.
- Acción: Mover un disco de una torre a otra, siempre y cuando éste no se apoye sobre un disco más chico.
- Frontera: Aquella combinación de posiciones que ya fueron tomadas por el agente versus las posiciones que todavia no fueron exploradas.

4. Implementación método de búsqueda. 

In [ ]:
class Node:
    def __init__(self, state, parent=None, cost=0, heuristic=0):
        self.state = state
        self.parent = parent
        self.cost = cost
        self.heuristic = heuristic

    def __lt__(self, other):
        return self.heuristic < other.heuristic


In [ ]:
def reconstruct_path(node):
    path = []
    while node:
        path.append(node.state)
        node = node.parent
    return path[::-1]


In [ ]:
def heuristic(state):
    goal_post = state[2]
    return len(goal_post) - sum(1 for i, disk in enumerate(goal_post) if disk == len(goal_post) - i)


In [ ]:
def get_neighbors(state):
    COST = 1
    neighbors = []
    for i in range(3):
        if state[i]:
            disk = state[i][-1]
            for j in range(3):
                if i != j and (not state[j] or state[j][-1] > disk):
                    new_state = [peg[:] for peg in state]
                    new_state[i].pop()
                    new_state[j].append(disk)
                    neighbors.append((new_state, COST))
    return neighbors


In [ ]:
import heapq

def search_function(start_state, goal_state, heuristic_func, get_neighbors, type = "greedy_best_first"):
    open_list = []
    heapq.heappush(open_list, Node(start_state, heuristic=heuristic_func(start_state)))
    closed_list = set()
    
    while open_list:
        current_node = heapq.heappop(open_list)
        
        if current_node.state == goal_state:
            return reconstruct_path(current_node)
        
        closed_list.add(tuple(tuple(peg) for peg in current_node.state))
        
        for neighbor, cost in get_neighbors(current_node.state):
            neighbor_tuple = tuple(tuple(peg) for peg in neighbor)
            if neighbor_tuple in closed_list:
                continue
            
            g_cost = current_node.cost + cost
            h_cost = heuristic_func(neighbor)
            if type == "greedy_best_first":
                f_cost = h_cost
            elif type == "a_star":
                f_cost = g_cost + h_cost
            
            neighbor_node = Node(neighbor, current_node, g_cost, f_cost)
            heapq.heappush(open_list, neighbor_node)
    
    return None

6. A nivel implementación, ¿qué tiempo y memoria ocupa el algoritmo? 

In [ ]:
%%timeit

start_state = [[5, 4, 3, 2, 1], [], []]
goal_state = [[], [], [5, 4, 3, 2, 1]]

path = search_function(start_state, goal_state, heuristic, get_neighbors,"a_star")


| Muestra | Tiempo (ms) |
|---------|-------------|
| Muestra 1 | 3.26        |
| Muestra 2 | 3.31        |
| Muestra 3 | 3.30        |
| Muestra 4 | 3.33        |
| Muestra 5 | 3.27        |
| Muestra 6 | 3.30        |
| Muestra 7 | 3.29        |
| Muestra 8 | 3.32        |
| Muestra 9 | 3.26        |
| Muestra 10 | 3.27        |
| **Promedio:** | **3.29 ms** |
| **Desviación Estándar:** | **0.028** |

Esta medición de tiempo de ejecución depende del hardware en el cual fue ejecutado


In [ ]:
import tracemalloc
tracemalloc.start()


start_state = [[5, 4, 3, 2, 1], [], []]
goal_state = [[], [], [5, 4, 3, 2, 1]]

path = search_function(start_state, goal_state, heuristic, get_neighbors, "a_star")
# for step in path:
    # print(step)

_, memory_peak = tracemalloc.get_traced_memory()
memory_peak /= 1024*1024
tracemalloc.stop()

print(f"Maxima memoria ocupada: {round(memory_peak, 2)} [MB]", )

| Muestra   | Tamaño [MB] |
|-----------|-------------|
| Muestra 1 | 0.12        |
| Muestra 2 | 0.11        |
| Muestra 3 | 0.12        |
| Muestra 4 | 0.11        |
| Muestra 5 | 0.11        |
| Muestra 6 | 0.11        |
| Muestra 7 | 0.11        |
| Muestra 8 | 0.11        |
| Muestra 9 | 0.11        |
| Muestra 10 | 0.11        |
| **Promedio:** | **0.113 MB** |
| **Desviación Estándar:** | **0.0078 MB** |


In [ ]:
import json

# Estados de las torres después de cada movimiento
states = path

# Lista de movimientos
movements = []

# Función para encontrar el disco movido y las posiciones
def find_movement(prev_state, curr_state):
    for i in range(3):  # Iterar sobre las tres torres
        if prev_state[i] != curr_state[i]:
            # Encuentra las torres de inicio y fin
            if len(prev_state[i]) < len(curr_state[i]):
                peg_end = i
            else:
                peg_start = i
    # Encuentra el disco movido
    disk = list(set(prev_state[peg_start]) - set(curr_state[peg_start]))[0]
    return {
        "type": "movement",
        "disk": disk,
        "peg_start": peg_start + 1,  # Convertir a 1-indexed
        "peg_end": peg_end + 1       # Convertir a 1-indexed
    }

# Generar los movimientos
for i in range(len(states) - 1):
    movement = find_movement(states[i], states[i + 1])
    movements.append(movement)

# Guardar en un archivo JSON
with open('../lib/intro_ia/clase2/hanoi_tower/simulator/sequence.json', 'w') as file:
    json.dump(movements, file, indent=4)

7. Cantidad de movimientos requeridos por el algoritmo

In [ ]:
mov_done = len(path) - 1
print("Movimientos : {}".format(mov_done))

mov_expected = 2**5 -1
print("Movimientos optimos : {}".format(mov_expected))

diff = abs(mov_done - mov_expected)
print("Diferencia entre el optimo y el implementado : {} ({}%)".format(diff, round(diff/mov_expected * 100),3))

In [ ]:
import os
import sys

# Guardar el directorio actual
current_dir = os.getcwd()

# Cambiar al directorio del script
target_dir = os.path.abspath(os.path.join(current_dir, '..', 'lib', 'intro_ia', 'clase2', 'hanoi_tower', 'simulator'))

os.chdir(target_dir)

# Asegurarse de que el directorio esté en sys.path
sys.path.append(target_dir)

# Ejecutar el script
%run simulation_hanoi.py

# Volver al directorio original
os.chdir(current_dir)